In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import networkx as nx
import random
import time
import pickle as cp
import sys
from tqdm import tqdm
import PrepareBatchGraph
import graph
import nstep_replay_mem
import nstep_replay_mem_prioritized
import mvc_env
import utils
import heapq
import scipy.linalg as linalg
import os
import pandas as pd

/Users/farazkhoshbakhtian/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from FINDER_torch import FINDER

In [3]:
dqn = FINDER()

('CUDA:', False)


In [4]:
dqn.Train()


generating validation graphs...


100%|███████████████████████████████████████████████████████████████████████████████████| 200/200 [00:10<00:00, 19.20it/s]

Validation of HDA: 0.4680187079082043
Validation of HBA: 0.4491250417937027

generating new training graphs...



100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1533.29it/s]
/var/folders/_t/ycs1gv8s5433r_f6cdcw7ny40000gn/T/ipykernel_99143/347219975.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dqn.Train()


----From fit: (dense dimensions of input tensors
('Action Select:', torch.Size([64, 1397]))
('rep_global:', torch.Size([0, 0]))
('n2nsum_param:', torch.Size([1397, 1397]))
('laplacian_param:', torch.Size([1397, 1397]))
('subgsum_param:', torch.Size([64, 1397]))
('(list) node_input:', 1397)
('(list) aux_input:', 64)
('(list) target:', 64)


AttributeError: 'FINDER' object has no attribute 'session'

In [ ]:
"""
rewrited tf to pytorch methods

old code is commented out as ref, for example:

#y_input_message = tf.matmul(tf.cast(y_node_input,tf.float32), w_n2l)
y_input_message = torch.matmul(self.y_node_input.type(torch.FloatTensor), w_n2l)

Prepared params such as:
y_nodes_size = tf.shape(self.subgsum_param)[0]

are kept as-is, since I want to test if this model will run at the end of rewrite functions and layers.

"""

In [35]:
class Model(nn.Module):
    def __init__(self, embedding_size=64, w_initialization_std=1, reg_hidden=32, max_bp_iter=3, embeddingMethod=1, aux_dim=4):
        super(Model, self).__init__()
        
        self.rand_generator = torch.normal
        self.embedding_size = embedding_size
        self.w_initialization_std = w_initialization_std
        self.reg_hidden = reg_hidden
        self.max_bp_iter = max_bp_iter
        self.embeddingMethod = embeddingMethod
        self.aux_dim = aux_dim
        
        self.act = nn.ReLU()
        
        # [2, embed_dim]
        self.w_n2l = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                     size=(2, self.embedding_size)))
        # [embed_dim, embed_dim]
        self.p_node_conv = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(self.embedding_size, self.embedding_size)))
        
        if self.embeddingMethod == 1:    #'graphsage'
            # [embed_dim, embed_dim]
            self.p_node_conv2 = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(self.embedding_size, self.embedding_size)))
            # [2*embed_dim, embed_dim]
            self.p_node_conv3 = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(2*self.embedding_size, self.embedding_size)))

        #[reg_hidden+aux_dim, 1]
        if self.reg_hidden > 0:
            # [embed_dim, reg_hidden]
            #h1_weight = tf.Variable(tf.truncated_normal([self.embedding_size, self.reg_hidden], stddev=initialization_stddev), tf.float32)
            self.h1_weight = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(self.embedding_size, self.reg_hidden)))
            
            #[reg_hidden+aux_dim, 1]
            #h2_weight = tf.Variable(tf.truncated_normal([self.reg_hidden + aux_dim, 1], stddev=initialization_stddev), tf.float32)
            self.h2_weight = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(self.reg_hidden + self.aux_dim, 1)))
            #[reg_hidden2 + aux_dim, 1]
            self.last_w = self.h2_weight
        else:
            #[2*embed_dim, reg_hidden]
            #h1_weight = tf.Variable(tf.truncated_normal([2 * self.embedding_size, self.reg_hidden], stddev=initialization_stddev), tf.float32)
            self.h1_weight = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(2*self.embedding_size, self.reg_hidden)))            
            #[2*embed_dim, reg_hidden]
            self.last_w = h1_weight
            
        ## [embed_dim, 1]
        #cross_product = tf.Variable(tf.truncated_normal([self.embedding_size, 1], stddev=initialization_stddev), tf.float32)
        self.cross_product = nn.parameter.Parameter(data=self.rand_generator(0, self.w_initialization_std,\
                                                                           size=(self.embedding_size, 1)))


    def train_forward(self, node_input, subgsum_param, n2nsum_param, action_select, aux_input):
        
        y_nodes_size = subgsum_param.shape[0]
        # [batch_size, 2]
        y_node_input = torch.ones((y_nodes_size,2))

        #[node_cnt, 2] * [2, embed_dim] = [node_cnt, embed_dim]
        # no sparse
        #input_message = tf.matmul(tf.cast(self.node_input,tf.float32), w_n2l)
        input_message = torch.matmul(node_input.type(torch.FloatTensor), self.w_n2l)
        #[node_cnt, embed_dim]  # no sparse
        #input_potential_layer = tf.nn.relu(input_message)
        input_potential_layer = self.act(input_message)

        # # no sparse
        # [batch_size, embed_dim]
        #y_input_message = tf.matmul(tf.cast(y_node_input,tf.float32), w_n2l)
        y_input_message = torch.matmul(y_node_input.type(torch.FloatTensor), self.w_n2l)
        #[batch_size, embed_dim]  # no sparse
        #y_input_potential_layer = tf.nn.relu(y_input_message)
        y_input_potential_layer = self.act(y_input_message)

        #input_potential_layer = input_message
        lv = 0
        #[node_cnt, embed_dim], no sparse
        cur_message_layer = input_potential_layer
        #cur_message_layer = tf.nn.l2_normalize(cur_message_layer, axis=1)
        cur_message_layer = torch.nn.functional.normalize(cur_message_layer, p=2, dim=1)

        #[batch_size, embed_dim], no sparse
        y_cur_message_layer = y_input_potential_layer
        # [batch_size, embed_dim]
        #y_cur_message_layer = tf.nn.l2_normalize(y_cur_message_layer, axis=1)
        y_cur_message_layer = torch.nn.functional.normalize(y_cur_message_layer, p=2, dim=1)

        # max_bp_iter=3  
        while lv < self.max_bp_iter:
            lv = lv + 1
            #[node_cnt, node_cnt] * [node_cnt, embed_dim] = [node_cnt, embed_dim], dense
            #n2npool = tf.sparse_tensor_dense_matmul(tf.cast(self.n2nsum_param,tf.float32), cur_message_layer)
            # see https://discuss.pytorch.org/t/sparse-tensors-in-pytorch/859/4
            n2npool = torch.matmul(n2nsum_param.type(torch.FloatTensor), cur_message_layer)
            #[node_cnt, embed_dim] * [embed_dim, embed_dim] = [node_cnt, embed_dim], dense
            #node_linear = tf.matmul(n2npool, p_node_conv)
            node_linear = torch.matmul(n2npool, self.p_node_conv)

            # [batch_size, node_cnt] * [node_cnt, embed_dim] = [batch_size, embed_dim]
            #y_n2npool = tf.sparse_tensor_dense_matmul(tf.cast(self.subgsum_param,tf.float32), cur_message_layer)
            # why cur_message_layer, instead of y_cur_message_layer?
            y_n2npool = torch.matmul(subgsum_param.type(torch.FloatTensor), cur_message_layer)
            #[batch_size, embed_dim] * [embed_dim, embed_dim] = [batch_size, embed_dim], dense
            #y_node_linear = tf.matmul(y_n2npool, p_node_conv)
            y_node_linear = torch.matmul(y_n2npool, self.p_node_conv)

            if self.embeddingMethod == 0: # 'structure2vec'
                #[node_cnt, embed_dim] + [node_cnt, embed_dim] = [node_cnt, embed_dim], return tensed matrix
                #merged_linear = tf.add(node_linear,input_message)
                merged_linear = torch.add(node_linear,input_message)
                #[node_cnt, embed_dim]
                #cur_message_layer = tf.nn.relu(merged_linear)
                cur_message_layer = self.act(merged_linear)

                #[batch_size, embed_dim] + [batch_size, embed_dim] = [batch_size, embed_dim], return tensed matrix
                #y_merged_linear = tf.add(y_node_linear, y_input_message)
                y_merged_linear = torch.add(y_node_linear, y_input_message)
                #[batch_size, embed_dim]
                #y_cur_message_layer = tf.nn.relu(y_merged_linear)
                y_cur_message_layer = self.act(y_merged_linear)
                
            else:   # 'graphsage'
                #[node_cnt, embed_dim] * [embed_dim, embed_dim] = [node_cnt, embed_dim], dense
                #cur_message_layer_linear = tf.matmul(tf.cast(cur_message_layer, tf.float32), p_node_conv2)
                cur_message_layer_linear = torch.matmul(cur_message_layer.type(torch.FloatTensor), self.p_node_conv2)
                #[[node_cnt, embed_dim] [node_cnt, embed_dim]] = [node_cnt, 2*embed_dim], return tensed matrix
                #merged_linear = tf.concat([node_linear, cur_message_layer_linear], 1)
                merged_linear = torch.concat([node_linear, cur_message_layer_linear], 1)
                #[node_cnt, 2*embed_dim]*[2*embed_dim, embed_dim] = [node_cnt, embed_dim]
                #cur_message_layer = tf.nn.relu(tf.matmul(merged_linear, p_node_conv3))
                cur_message_layer = self.act(torch.matmul(merged_linear, self.p_node_conv3))

                #[batch_size, embed_dim] * [embed_dim, embed_dim] = [batch_size, embed_dim], dense
                #y_cur_message_layer_linear = tf.matmul(tf.cast(y_cur_message_layer, tf.float32), p_node_conv2)
                y_cur_message_layer_linear = torch.matmul(y_cur_message_layer.type(torch.FloatTensor), self.p_node_conv2)
                #[[batch_size, embed_dim] [batch_size, embed_dim]] = [batch_size, 2*embed_dim], return tensed matrix
                #y_merged_linear = tf.concat([y_node_linear, y_cur_message_layer_linear], 1)
                y_merged_linear = torch.concat([y_node_linear, y_cur_message_layer_linear], 1)
                #[batch_size, 2*embed_dim]*[2*embed_dim, embed_dim] = [batch_size, embed_dim]
                #y_cur_message_layer = tf.nn.relu(tf.matmul(y_merged_linear, p_node_conv3))
                y_cur_message_layer = self.act(torch.matmul(y_merged_linear, self.p_node_conv3))

            #cur_message_layer = tf.nn.l2_normalize(cur_message_layer, axis=1)
            #y_cur_message_layer = tf.nn.l2_normalize(y_cur_message_layer, axis=1)
            cur_message_layer = torch.nn.functional.normalize(cur_message_layer, p=2, dim=1)
            y_cur_message_layer = torch.nn.functional.normalize(y_cur_message_layer, p=2, dim=1)

        
        node_embedding = cur_message_layer
        #[batch_size, node_cnt] * [node_cnt, embed_dim] = [batch_size, embed_dim], dense
        y_potential = y_cur_message_layer
        #[batch_size, node_cnt] * [node_cnt, embed_dim] = [batch_size, embed_dim]
        #action_embed = tf.sparse_tensor_dense_matmul(tf.cast(self.action_select, tf.float32), cur_message_layer)
        action_embed = torch.matmul(action_select.type(torch.FloatTensor), cur_message_layer)

        # # [batch_size, embed_dim, embed_dim]
        #temp = tf.matmul(tf.expand_dims(action_embed, axis=2),tf.expand_dims(y_potential, axis=1))
        temp = torch.matmul(torch.unsqueeze(action_embed, dim=2),torch.unsqueeze(y_potential, dim=1))
        # [batch_size, embed_dim]
        #Shape = tf.shape(action_embed)
        Shape = action_embed.size()
        # [batch_size, embed_dim], first transform
        #embed_s_a = tf.reshape(tf.matmul(temp, tf.reshape(tf.tile(cross_product,[Shape[0],1]),[Shape[0],Shape[1],1])),Shape)
        embed_s_a = torch.reshape(torch.matmul(temp, torch.reshape(torch.tile(self.cross_product,[Shape[0],1]),\
                                                                   [Shape[0],Shape[1],1])),Shape)

        #[batch_size, 2 * embed_dim]
        last_output = embed_s_a

        if self.reg_hidden > 0:
            #[batch_size, 2*embed_dim] * [2*embed_dim, reg_hidden] = [batch_size, reg_hidden], dense
            #hidden = tf.matmul(embed_s_a, h1_weight)
            hidden = torch.matmul(embed_s_a, self.h1_weight)
            #[batch_size, reg_hidden]
            #last_output = tf.nn.relu(hidden)
            last_output = self.act(hidden)

        # if reg_hidden == 0: ,[[batch_size, 2*embed_dim], [batch_size, aux_dim]] = [batch_size, 2*embed_dim+aux_dim]
        # if reg_hidden > 0: ,[[batch_size, reg_hidden], [batch_size, aux_dim]] = [batch_size, reg_hidden+aux_dim]
        #last_output = tf.concat([last_output, self.aux_input], 1)
        last_output = torch.concat([last_output, aux_input], 1)
        #if reg_hidden == 0: ,[batch_size, 2*embed_dim+aux_dim] * [2*embed_dim+aux_dim, 1] = [batch_size, 1]
        #if reg_hidden > 0: ,[batch_size, reg_hidden+aux_dim] * [reg_hidden+aux_dim, 1] = [batch_size, 1]
        #q_pred = tf.matmul(last_output, last_w)
        q_pred = torch.matmul(last_output, self.last_w)
        
        ## first order reconstruction loss
        #loss_recons = 2 * torch.trace(torch.matmul(torch.transpose(cur_message_layer), torch.matmul(laplacian_param.type(torch.FloatTensor), cur_message_layer)))
        #edge_num =  torch.sparse.sum(self.n2nsum_param)
        #loss_recons = tf.divide(loss_recons, edge_num)


        # TODO, losses and more
        
        return q_pred, cur_message_layer
    
    def test_forward(self, node_input, subgsum_param, n2nsum_param, rep_global, aux_input):
        
        y_nodes_size = subgsum_param.shape[0]
        # [batch_size, 2]
        y_node_input = torch.ones((y_nodes_size,2))

        #[node_cnt, 2] * [2, embed_dim] = [node_cnt, embed_dim]
        # no sparse
        #input_message = tf.matmul(tf.cast(self.node_input,tf.float32), w_n2l)
        input_message = torch.matmul(node_input.type(torch.FloatTensor), self.w_n2l)
        #[node_cnt, embed_dim]  # no sparse
        #input_potential_layer = tf.nn.relu(input_message)
        input_potential_layer = self.act(input_message)

        # # no sparse
        # [batch_size, embed_dim]
        #y_input_message = tf.matmul(tf.cast(y_node_input,tf.float32), w_n2l)
        y_input_message = torch.matmul(y_node_input.type(torch.FloatTensor), self.w_n2l)
        #[batch_size, embed_dim]  # no sparse
        #y_input_potential_layer = tf.nn.relu(y_input_message)
        y_input_potential_layer = self.act(y_input_message)

        #input_potential_layer = input_message
        lv = 0
        #[node_cnt, embed_dim], no sparse
        cur_message_layer = input_potential_layer
        #cur_message_layer = tf.nn.l2_normalize(cur_message_layer, axis=1)
        cur_message_layer = torch.nn.functional.normalize(cur_message_layer, p=2, dim=1)

        #[batch_size, embed_dim], no sparse
        y_cur_message_layer = y_input_potential_layer
        # [batch_size, embed_dim]
        #y_cur_message_layer = tf.nn.l2_normalize(y_cur_message_layer, axis=1)
        y_cur_message_layer = torch.nn.functional.normalize(y_cur_message_layer, p=2, dim=1)

        # max_bp_iter=3  
        while lv < self.max_bp_iter:
            lv = lv + 1
            #[node_cnt, node_cnt] * [node_cnt, embed_dim] = [node_cnt, embed_dim], dense
            #n2npool = tf.sparse_tensor_dense_matmul(tf.cast(self.n2nsum_param,tf.float32), cur_message_layer)
            # see https://discuss.pytorch.org/t/sparse-tensors-in-pytorch/859/4
            n2npool = torch.matmul(n2nsum_param.type(torch.FloatTensor), cur_message_layer)
            #[node_cnt, embed_dim] * [embed_dim, embed_dim] = [node_cnt, embed_dim], dense
            #node_linear = tf.matmul(n2npool, p_node_conv)
            node_linear = torch.matmul(n2npool, self.p_node_conv)

            # [batch_size, node_cnt] * [node_cnt, embed_dim] = [batch_size, embed_dim]
            #y_n2npool = tf.sparse_tensor_dense_matmul(tf.cast(self.subgsum_param,tf.float32), cur_message_layer)
            # why cur_message_layer, instead of y_cur_message_layer?
            y_n2npool = torch.matmul(subgsum_param.type(torch.FloatTensor), cur_message_layer)
            #[batch_size, embed_dim] * [embed_dim, embed_dim] = [batch_size, embed_dim], dense
            #y_node_linear = tf.matmul(y_n2npool, p_node_conv)
            y_node_linear = torch.matmul(y_n2npool, self.p_node_conv)

            if self.embeddingMethod == 0: # 'structure2vec'
                #[node_cnt, embed_dim] + [node_cnt, embed_dim] = [node_cnt, embed_dim], return tensed matrix
                #merged_linear = tf.add(node_linear,input_message)
                merged_linear = torch.add(node_linear,input_message)
                #[node_cnt, embed_dim]
                #cur_message_layer = tf.nn.relu(merged_linear)
                cur_message_layer = self.act(merged_linear)

                #[batch_size, embed_dim] + [batch_size, embed_dim] = [batch_size, embed_dim], return tensed matrix
                #y_merged_linear = tf.add(y_node_linear, y_input_message)
                y_merged_linear = torch.add(y_node_linear, y_input_message)
                #[batch_size, embed_dim]
                #y_cur_message_layer = tf.nn.relu(y_merged_linear)
                y_cur_message_layer = self.act(y_merged_linear)
                
            else:   # 'graphsage'
                #[node_cnt, embed_dim] * [embed_dim, embed_dim] = [node_cnt, embed_dim], dense
                #cur_message_layer_linear = tf.matmul(tf.cast(cur_message_layer, tf.float32), p_node_conv2)
                cur_message_layer_linear = torch.matmul(cur_message_layer.type(torch.FloatTensor), self.p_node_conv2)
                #[[node_cnt, embed_dim] [node_cnt, embed_dim]] = [node_cnt, 2*embed_dim], return tensed matrix
                #merged_linear = tf.concat([node_linear, cur_message_layer_linear], 1)
                merged_linear = torch.concat([node_linear, cur_message_layer_linear], 1)
                #[node_cnt, 2*embed_dim]*[2*embed_dim, embed_dim] = [node_cnt, embed_dim]
                #cur_message_layer = tf.nn.relu(tf.matmul(merged_linear, p_node_conv3))
                cur_message_layer = self.act(torch.matmul(merged_linear, self.p_node_conv3))

                #[batch_size, embed_dim] * [embed_dim, embed_dim] = [batch_size, embed_dim], dense
                #y_cur_message_layer_linear = tf.matmul(tf.cast(y_cur_message_layer, tf.float32), p_node_conv2)
                y_cur_message_layer_linear = torch.matmul(y_cur_message_layer.type(torch.FloatTensor), self.p_node_conv2)
                #[[batch_size, embed_dim] [batch_size, embed_dim]] = [batch_size, 2*embed_dim], return tensed matrix
                #y_merged_linear = tf.concat([y_node_linear, y_cur_message_layer_linear], 1)
                y_merged_linear = torch.concat([y_node_linear, y_cur_message_layer_linear], 1)
                #[batch_size, 2*embed_dim]*[2*embed_dim, embed_dim] = [batch_size, embed_dim]
                #y_cur_message_layer = tf.nn.relu(tf.matmul(y_merged_linear, p_node_conv3))
                y_cur_message_layer = self.act(torch.matmul(y_merged_linear, self.p_node_conv3))

            #cur_message_layer = tf.nn.l2_normalize(cur_message_layer, axis=1)
            #y_cur_message_layer = tf.nn.l2_normalize(y_cur_message_layer, axis=1)
            cur_message_layer = torch.nn.functional.normalize(cur_message_layer, p=2, dim=1)
            y_cur_message_layer = torch.nn.functional.normalize(y_cur_message_layer, p=2, dim=1)

        
            y_potential = y_cur_message_layer        #[node_cnt, batch_size] * [batch_size, embed_dim] = [node_cnt, embed_dim]
        
        rep_y = torch.matmul(rep_global.type(torch.FloatTensor), y_potential)
        #[[node_cnt, embed_dim], [node_cnt, embed_dim]] = [node_cnt, 2*embed_dim]
        # embed_s_a_all = tf.concat([cur_message_layer,rep_y],1)
        # # [node_cnt, embed_dim, embed_dim]
        temp1 = torch.matmul(torch.unsqueeze(cur_message_layer, dim=2),torch.unsqueeze(rep_y, dim=1))
        # [node_cnt embed_dim]
        Shape1 = cur_message_layer.size()
        # [batch_size, embed_dim], first transform
        embed_s_a_all = torch.reshape(torch.matmul(temp1, torch.reshape(torch.tile(self.cross_product,[Shape1[0],1]),[Shape1[0],Shape1[1],1])),Shape1)

        #[node_cnt, 2 * embed_dim]
        last_output = embed_s_a_all
        if self.reg_hidden > 0:
            #[node_cnt, 2 * embed_dim] * [2 * embed_dim, reg_hidden] = [node_cnt, reg_hidden1]
            hidden = torch.matmul(embed_s_a_all, self.h1_weight)
            #Relu, [node_cnt, reg_hidden1]
            last_output = self.act(hidden)
            #[node_cnt, reg_hidden1] * [reg_hidden1, reg_hidden2] = [node_cnt, reg_hidden2]
            # last_output_hidden = tf.matmul(last_output1, h2_weight)
            # last_output = tf.nn.relu(last_output_hidden)

        #[node_cnt, batch_size] * [batch_size, aux_dim] = [node_cnt, aux_dim]
        rep_aux = torch.matmul(rep_global.type(torch.FloatTensor), aux_input)

        #if reg_hidden == 0: , [[node_cnt, 2 * embed_dim], [node_cnt, aux_dim]] = [node_cnt, 2*embed_dim + aux_dim]
        #if reg_hidden > 0: , [[node_cnt, reg_hidden], [node_cnt, aux_dim]] = [node_cnt, reg_hidden + aux_dim]
        last_output = torch.concat([last_output,rep_aux],1)

        #if reg_hidden == 0: , [node_cnt, 2 * embed_dim + aux_dim] * [2 * embed_dim + aux_dim, 1] = [node_cnt，1]
        #f reg_hidden > 0: , [node_cnt, reg_hidden + aux_dim] * [reg_hidden + aux_dim, 1] = [node_cnt，1]
        q_on_all = torch.matmul(last_output, self.last_w)


        return q_on_all

In [36]:
## Training loss for later:

## first order reconstruction loss
#loss_recons = 2 * torch.trace(torch.matmul(torch.transpose(cur_message_layer), torch.matmul(laplacian_param.type(torch.FloatTensor), cur_message_layer)))
#edge_num =  torch.sparse.sum(n2nsum_param)
#loss_recons = tf.divide(loss_recons, edge_num)

'''
if self.IsPrioritizedSampling:
    self.TD_errors = tf.reduce_sum(tf.abs(self.target - q_pred), axis=1)    # for updating Sumtree
    if self.IsHuberloss:
        loss_rl = tf.losses.huber_loss(self.ISWeights * self.target, self.ISWeights * q_pred)
    else:
        loss_rl = tf.reduce_mean(self.ISWeights * tf.squared_difference(self.target, q_pred))
else:
    if self.IsHuberloss:
        loss_rl = tf.losses.huber_loss(self.target, q_pred)
    else:
        loss_rl = tf.losses.mean_squared_error(self.target, q_pred)

loss = loss_rl + Alpha * loss_recons
'''
i=1

In [37]:
M = Model()

In [38]:
type(dqn.inputs['subgsum_param'])

torch.Tensor

In [39]:
pytorch_total_params = sum(p.numel() for p in M.parameters())
print(pytorch_total_params)

18660


In [40]:
node_input = torch.tensor(dqn.inputs['node_input'])
subgsum_param = torch.tensor(dqn.inputs['subgsum_param'])
n2nsum_param = torch.tensor(dqn.inputs['n2nsum_param'])
action_select = torch.tensor(dqn.inputs['action_select'])
aux_input = torch.tensor(dqn.inputs['aux_input'])
rep_global = torch.tensor(dqn.inputs['rep_global'])

/var/folders/_t/ycs1gv8s5433r_f6cdcw7ny40000gn/T/ipykernel_99143/1475947402.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  subgsum_param = torch.tensor(dqn.inputs['subgsum_param'])
/var/folders/_t/ycs1gv8s5433r_f6cdcw7ny40000gn/T/ipykernel_99143/1475947402.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  n2nsum_param = torch.tensor(dqn.inputs['n2nsum_param'])
/var/folders/_t/ycs1gv8s5433r_f6cdcw7ny40000gn/T/ipykernel_99143/1475947402.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  action_select = torch.tensor(dqn.inputs['actio

In [41]:
print(node_input.shape)
print(subgsum_param.shape)
print(n2nsum_param.shape)
print(action_select.shape)
print(aux_input.shape)
rep_global

torch.Size([1397, 2])
torch.Size([64, 1397])
torch.Size([1397, 1397])
torch.Size([64, 1397])
torch.Size([64, 4])


tensor(indices=tensor([], size=(2, 0)),
       values=tensor([], size=(0,)),
       size=(0, 0), nnz=0, dtype=torch.float64, layout=torch.sparse_coo)

In [42]:
q_pred, cur_message_layer = M.train_forward(node_input, subgsum_param, n2nsum_param, action_select, aux_input)
print(q_pred.shape)
print(cur_message_layer.shape)

torch.Size([64, 1])
torch.Size([1397, 64])


In [43]:
# in actual testing scnario, instead of action_select, the model recives a variable called rep_global
temp =torch.transpose(action_select,0,1)

In [44]:
q_pred_all = M.test_forward(node_input, subgsum_param, n2nsum_param, temp, aux_input)
print(q_pred_all.shape)


torch.Size([1397, 1])


In [ ]:
sample = dqn.nStepReplayMem.Sampling(64)

In [44]:
sample

## Misc

In [4]:
x_a, y_a = torch.ones((100, 2)), torch.ones(100)
x_b, y_b = torch.zeros((100, 2)), torch.zeros(100)


In [10]:
for i in range (4):
    
    M = Model(1, 1)


    numb_parameters = 0
    for param in M.parameters():
        numb_parameters += np.prod(list(param.shape))
    print(numb_parameters)

    print(M(torch.ones((5, 2))))



3
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)
3
tensor([[0.8910],
        [0.8910],
        [0.8910],
        [0.8910],
        [0.8910]], grad_fn=<ReluBackward0>)
3
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)
3
tensor([[0.1697],
        [0.1697],
        [0.1697],
        [0.1697],
        [0.1697]], grad_fn=<ReluBackward0>)


In [6]:
optimizer = optim.SGD(M.parameters(), lr=0.01, momentum=0.9)
criterion = nn.BCEWithLogitsLoss()


In [7]:
for i in range(100):
    optimizer.zero_grad()
    # Forward pass, backward pass, and optimize
    outputs = M(x_a)
    loss = criterion(outputs[:,0], y_a.float())
    loss.backward()
    optimizer.step()
    
    optimizer.zero_grad()
    # Forward pass, backward pass, and optimize
    outputs = M(x_b)
    loss = criterion(outputs[:,0], y_b.float())
    loss.backward()
    optimizer.step()

In [8]:
M(torch.ones(20,2))

tensor([[3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784],
        [3.0784]], grad_fn=<ReluBackward0>)

In [9]:
M(torch.zeros(20,2))

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)